In [1]:
import ROOT
import math
import numpy as np

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)

def calculate_delta_phi_jet_met(jet_phi, met_phi): 
    return delta_phi(jet_phi, met_phi)

def analyze_specific_points(root_file_path, cross_section):
    # Create chain of root trees
    chain = ROOT.TChain("Delphes")
    chain.Add(root_file_path)

    # Create object of class ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(chain)
    numberOfEntries = treeReader.GetEntries()

    # Get pointer to the branch containing Particle information
    branchJet = treeReader.UseBranch("Jet")
    branchMET = treeReader.UseBranch("MissingET")
    branchEvent = treeReader.UseBranch("Weight")

    # Events Counter
    initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

    #### Initial Cuts 
    pt_cut = 30.0
    max_eta = 2.8  

    # Constants for scaling
    luminosity = 140.0 * 1000  # pb
    # cross_section = 0.385  # pb for gluino at 1 TeV

    # Dictionary to store MET cut values and corresponding event counts
    met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

    sum_event_weights = 0
    for i in range(numberOfEntries):
        treeReader.ReadEntry(i)
        sum_event_weights += branchEvent.At(6).Weight

    # Loop over all events
    for entry in range(numberOfEntries):
        # Load selected branches with data from specified event
        treeReader.ReadEntry(entry)

        num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

        initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights

        if branchMET.At(0).MET > 200:
            events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

            # Loop over all jets
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)

                # Jet Pt and Eta Cuts
                if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                    num_jets_meeting_criteria += 1

                # Save Leading Jet Pt and Eta
                if jet.PT > leading_jet_pt:
                    leading_jet_pt = jet.PT
                    leading_jet_eta = jet.Eta

            # Count events passing pt, eta cut
            if num_jets_meeting_criteria > 0:
                events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

            # To set limit over jets
            if num_jets_meeting_criteria <= 4:
                events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

                # Delta_Phi cut implementation
                delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
                delta_phis = []
                for jet_index in range(branchJet.GetEntries()):
                    jet = branchJet.At(jet_index)
                    delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                    delta_phis.append(delta_phi_jet_met)

                # Check how many jets pass the delta_phi cut
                if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                    events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

                    # Leading Jets Pt and Eta cuts
                    if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                        events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

                        # Loop over MET cut values
                        for met_cut in met_cuts:
                            if branchMET.At(0).MET > met_cut:
                                met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

    print(f"\nAnalysis results for root file: {root_file_path}")
    print("===================================================")
    print(f"\nInitial number of events: {initial_events:.2f}")
    print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f} +/- {np.sqrt(events_passed_met_cut):.2f}")
    print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f} +/- {np.sqrt(events_passed_pt_eta_cut):.2f}")
    print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f} +/- {np.sqrt(events_passed_jets_criteria):.2f}")
    print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f} +/- {np.sqrt(events_passed_delta_phi_cut):.2f}")
    print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f} +/- {np.sqrt(events_passed_leading_jet_cut):.2f}")

    paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                      600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

    ratios = {}  # Dictionary to store the ratios
    
    # Calculate the ratio for each MET cut value
    for met_cut in met_cuts:
        if met_cut in paper_met_cuts:
            ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
            # ratio_uncertainty =         sqrt(  numerator_uncertainty / numerator)**2 + (denominator_uncertainty / denominator)**2)
            # ratio_uncertainty = ratio * sqrt(    (sqrt(numerator)    / numerator)**2 + (    sqrt(denominator)   / denominator)**2)
            # Numerator corresponds to met_cuts[met_cut]
            # Denominator corresponds to paper_met_cuts[met_cut]
            ratio_uncertainty = ratio * np.sqrt((np.sqrt(met_cuts[met_cut]) / met_cuts[met_cut]) ** 2 + (np.sqrt(paper_met_cuts[met_cut]) / paper_met_cuts[met_cut]) ** 2)
            ratios[met_cut] = (ratio, ratio_uncertainty)
            #ratios[met_cut] = ratio
            print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f} +/- {np.sqrt(met_cuts[met_cut]):.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f} +/- {ratio_uncertainty:.2f}")

    # max_ratio_met_cut = max(ratios, key=ratios.get)
    max_ratio_met_cut = max(ratios, key=lambda x: ratios[x][0])

    gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])

    print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)


Welcome to JupyROOT 6.26/06


# For N1 950 ; gl 1000 GeV ; using MLM
#### ### it has entry at 0 for branchEvent.At(6).Weight ; size of 4

In [2]:
# analyze_specific_points("root_files/MLM_N950/tag_1_delphes_events.root", 0.385)

# For N1 850 GeV ; gl 1000 GeV ; using CKKW matching so on

In [3]:
analyze_specific_points("root_files/N850/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N850/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13352.51 +/- 115.55

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13315.07 +/- 115.39

Number of events passed number of jets <= 4 cut: 11369.84 +/- 106.63

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9067.61 +/- 95.22

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7924.95 +/- 89.02

For SR MET > 200 GeV: S_Exp: 7924.95 +/- 89.02, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6255.05 +/- 79.09, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4744.78 +/- 68.88, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3606.94 +/- 60.06, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2735.73 +/- 52.30, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1695.81 +/- 41.18, S_

# For N1 900 GeV ; gl 1000 GeV

In [4]:
analyze_specific_points("root_files/N900/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N900/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13543.82 +/- 116.38

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13514.66 +/- 116.25

Number of events passed number of jets <= 4 cut: 11611.05 +/- 107.75

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9319.41 +/- 96.54

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8106.51 +/- 90.04

For SR MET > 200 GeV: S_Exp: 8106.51 +/- 90.04, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6305.28 +/- 79.41, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4679.32 +/- 68.41, S_Obs: 20893, and ratio: 0.22 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3627.50 +/- 60.23, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2753.52 +/- 52.47, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1652.47 +/- 40.65, S_

# For N1 950 GeV ; gl 1000 GeV 

In [5]:
analyze_specific_points("root_files/N950/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N950/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13572.21 +/- 116.50

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13534.77 +/- 116.34

Number of events passed number of jets <= 4 cut: 11561.63 +/- 107.53

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9220.30 +/- 96.02

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8087.94 +/- 89.93

For SR MET > 200 GeV: S_Exp: 8087.94 +/- 89.93, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6222.38 +/- 78.88, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4693.33 +/- 68.51, S_Obs: 20893, and ratio: 0.22 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3604.89 +/- 60.04, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2672.19 +/- 51.69, S_Obs: 7214, and ratio: 0.37 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1657.55 +/- 40.71, S_

# For N1 990 GeV ; gl 1000 GeV 

In [6]:
analyze_specific_points("root_files/N990/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N990/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13513.44 +/- 116.25

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13484.28 +/- 116.12

Number of events passed number of jets <= 4 cut: 11546.86 +/- 107.46

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9240.27 +/- 96.13

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8068.51 +/- 89.82

For SR MET > 200 GeV: S_Exp: 8068.51 +/- 89.82, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6250.29 +/- 79.06, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4758.78 +/- 68.98, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3586.75 +/- 59.89, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2754.19 +/- 52.48, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1696.33 +/- 41.19, S_

# For N1 990 GeV ; gl 1000 GeV ; again

In [7]:
analyze_specific_points("root_files/N990_again/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N990_again/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13598.09 +/- 116.61

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13575.67 +/- 116.51

Number of events passed number of jets <= 4 cut: 11585.94 +/- 107.64

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9364.53 +/- 96.77

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8146.51 +/- 90.26

For SR MET > 200 GeV: S_Exp: 8146.51 +/- 90.26, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6328.91 +/- 79.55, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4799.54 +/- 69.28, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3569.77 +/- 59.75, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2739.76 +/- 52.34, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1699.26 +/- 41.